# SIRS with mobility and network structure: SIMULATION SF vs SF

We fixed the epidemic parameters in a "critical" point at which most of the times the epidemics does not survive and dies out after some iterations. The topology of the two networks is also kept fixed. We now explore the role of the mobility parameter p_mob. In this case simulation is done coupling two scale free networks.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt 
import os
import networkx as nx
import pandas as pd
import time 
# to install tqdm
#! pip install tqdm
from tqdm import tnrange, trange

# import custom module
import SIRS_twoSF

In [ ]:
# needed only if you made some changes at SIRS_twoSF.py and want to import the updated version of the module
from importlib import reload
reload(SIRS_twoSF)

In [ ]:
#mobility and topology parameters
N = 1000
I_sf1_init = 5
I_sf2_init = 0
mean_degree = 4

In [ ]:
beta0 = 0.3
gamma = 0.016
mu = 0.15

p_mob_list = np.linspace(0.03,0.2,20)

print('Number of travellers:')
for p in p_mob_list:
    print('p=%.2f    N=%d'%(p, int(p*N)))

In [7]:
beta = beta0/mean_degree
infection_params = dict(beta=beta, mu=mu, gamma=gamma)


UP_directory_name = 'data_SF_asym/'+str(beta)+"_"+str(mu)+"_"+str(gamma) 
os.system("mkdir data_SF_asym")
os.system("mkdir "+ UP_directory_name)
start = time.time()

for j, p_mob in enumerate(p_mob_list):

    print('Simulation %d/%d'%(j+1,len(p_mob_list)), end =' - ')
    print("P_mob: %.3f"% p_mob)


    #saving stuff for future analysis
    directory_name = UP_directory_name+"/pmob_"+"{:3f}".format(p_mob)
    os.system("mkdir "+directory_name)

    #saving epidemics, topology and mobility parameters

    params=dict(N=N, I_sf1_init=I_sf1_init, I_sf2_nit=I_sf2_init, p_mob=p_mob, mean_degree=mean_degree,
                eps=eps, **infection_params)

    parameters = pd.DataFrame(params, index=[0])
    save=directory_name+"/parameters.csv"
    parameters.to_csv(save)



    #STARTING SIMULATION
    n_runs = 50
    n_iter = 1000



    for run in tnrange(n_runs, desc='1st loop', leave=True):


        # prepare systems
        state_sf1_init, state_sf2_init, variables_net_sf1, variables_net_sf2 = SIRS_twoSF.prepare_two_sys(N, I_sf1_init, I_sf2_init, 
                                                                                      p_mob, mean_degree)

        #saving initial states
        save = directory_name + "/state_sf1_init_"+str(run)+".txt"
        np.savetxt(save, state_sf1_init.astype(int), fmt ='%d')

        save = directory_name + "/state_sf2_init_"+str(run)+".txt"
        np.savetxt(save, state_sf1_init.astype(int), fmt ='%d')

        save = directory_name + "/travellers_sf1_"+str(run)+".txt"
        travellers_sf1 = variables_net_sf1["travellers_sf1"]
        np.savetxt(save, travellers_sf1.astype(int), fmt ='%d')
        
        save = directory_name + "/travellers_sf2_"+str(run)+".txt"
        travellers_sf2 = variables_net_sf2["travellers_sf2"]
        np.savetxt(save, travellers_sf2.astype(int), fmt ='%d')
        
        #saving initial variables: we just save the IDs of the travelling nodes and their degrees
        save = directory_name + "/deg_trav_sf1_"+str(run)+".txt"
        deg_sf1 = variables_net_sf1["A_sf1"].sum(axis=1)
        deg_trav_sf1 = deg_sf1[travellers_sf1]
        np.savetxt(save, deg_trav_sf1.astype(int), fmt ='%d')
        
        save = directory_name + "/deg_trav_sf2_"+str(run)+".txt"
        deg_sf2 = variables_net_sf2["A_sf2"].sum(axis=1)
        deg_trav_sf2 = deg_sf2[travellers_sf2]
        np.savetxt(save, deg_trav_sf2.astype(int), fmt ='%d')


        
        

        #print("Iteration:", run)

        state_sf1 = np.copy(state_sf1_init)
        state_sf2 = np.copy(state_sf2_init)

        S_sf1 = [N - I_sf1_init]
        I_sf1 = [I_sf1_init]
        R_sf1 = [0]
        S_sf2 = [N - I_sf2_init]
        I_sf2 = [I_sf2_init]
        R_sf2 = [0]
        t_vec = []
        for i in tnrange(n_iter, desc='2nd loop', leave=False):
            state_sf1, state_sf2 = SIRS_twoSF.two_sys_full_SIRS_step(state_sf1, state_sf2, **variables_net_sf1, 
                                                    **variables_net_sf2, **infection_params)
            S_sf1.append(state_sf1[:,0].sum())
            I_sf1.append(state_sf1[:,1].sum())
            R_sf1.append(state_sf1[:,2].sum())
            S_sf2.append(state_sf2[:,0].sum())
            I_sf2.append(state_sf2[:,1].sum())
            R_sf2.append(state_sf2[:,2].sum())
            #t_vec.append(time.time()-start)

        #tot_time = time.time()-start
        #print("Total time elapsed: %.2f s"%tot_time)
        #print("Time per iteration: %.4f s"%(tot_time/n_iter))

        S_sf1 = np.array(S_sf1)
        I_sf1 = np.array(I_sf1)
        R_sf1 = np.array(R_sf1)
        S_sf2 = np.array(S_sf2)
        I_sf2 = np.array(I_sf2)
        R_sf2 = np.array(R_sf2)
        t_vec = np.array(t_vec)

        #save timeseries of SIR
        SIR_sf1 = np.vstack((S_sf1, I_sf1, R_sf1)).T
        save = directory_name+"/SIR_sf1_"+str(run)+".txt"
        np.savetxt(save, SIR_sf1.astype(int), fmt ='%d')
        SIR_sf2 = np.vstack((S_sf2, I_sf2, R_sf2)).T
        save = directory_name+"/SIR_sf2_"+str(run)+".txt"
        np.savetxt(save, SIR_sf2.astype(int), fmt ='%d')

